# MENACE Game: Explanation

This code implements a simplified version of the MENACE (Machine Educable Noughts and Crosses Engine) model for playing Tic-Tac-Toe. MENACE uses an epsilon-greedy algorithm to choose actions, updates its model based on the result of each game, and saves and loads the model state using Python's `pickle` module.

## MENACE Class

### `__init__(self, model_file="menace_model.pkl")`
The constructor method initializes the MENACE object and attempts to load an existing model from a file. If the model does not exist, it starts with a fresh model.

- **`self.states`**: A `defaultdict` of `defaultdict(int)` to store state-action pairs and their corresponding weights. Each state-action pair represents the possible moves MENACE can make in the game.
- **`self.epsilon`**: A parameter used for the epsilon-greedy algorithm, where the action is chosen randomly with a probability of `epsilon`, promoting exploration.
- **`self.model_file`**: The file name where the model (state-action weights) is saved.

### `choose_action(self, state)`
This method selects an action based on the epsilon-greedy strategy. The action is chosen from valid moves in the current state.

- If a random value is less than `epsilon`, MENACE explores by selecting a random valid action.
- Otherwise, MENACE exploits its learned strategy by selecting the action with the highest weight in the current state.

### `update_weights(self, history, result)`
This method updates the state-action weights based on the result of the game.

- **`history`**: A list of tuples containing the states and actions MENACE took during the game.
- **`result`**: A string representing the game outcome (`"win"`, `"loss"`, or `"draw"`).
  
If MENACE wins, the state-action weights are increased for the moves that led to the win. If MENACE loses, the weights are decreased for the moves that led to the loss.

### `get_valid_actions(state)`
This static method returns a list of valid actions (empty cells) from the current game state. It identifies cells in the Tic-Tac-Toe grid where there is no move (i.e., where the cell value is `0`).

### `check_winner(state)`
This static method checks the winner of the game by evaluating all possible winning combinations in the Tic-Tac-Toe grid.

- If a winning combination exists with all `X` (MENACE's moves), it returns `"MENACE"`.
- If a winning combination exists with all `O` (User's moves), it returns `"User"`.
- If no winner is found, it returns `None`.

### `print_board(state)`
This static method prints the Tic-Tac-Toe board based on the current state. It converts the internal state (which is a list of integers) to human-readable symbols:
- `1` -> `"X"` (MENACE's move)
- `-1` -> `"O"` (User's move)
- `0` -> `" "` (empty cell)

### `play_game(self)`
This method runs the main game loop where MENACE plays against the user.

1. **MENACE's Turn**: MENACE selects an action using `choose_action` and updates the state by placing its symbol (`"X"`) in the selected cell. The game board is printed.
2. **Check for Winner**: After MENACE plays, the game checks if MENACE or the user has won or if the game is a draw.
3. **User's Turn**: The user is prompted to enter a valid move (a number between 0 and 8). The game ensures that the move is valid (i.e., an empty cell).
4. **Check for Winner**: After the user plays, the game checks if the user has won.

The game continues until there is a winner or a draw. After each game, the model is updated, and the model's state is saved to disk using `save_model()`.

### `save_model(self)`
This method saves the current state-action weights of MENACE to a file using `pickle`. The model is stored as a serialized object, allowing it to be loaded again in future sessions.

### `load_model(self)`
This method loads the saved model from a file. If the file does not exist (i.e., this is the first time running the game), it starts with an empty model.

- **`pickle.load(f)`**: Reads the model from the file and loads it into `self.states`.
- If the file is not found, it prints a message and initializes the model from scratch.

## Main Loop

```python
if __name__ == "__main__":
    menace = MENACE()
    print("Welcome to MENACE Tic-Tac-Toe!")
    print("You are 'O'. MENACE is 'X'. Enter a number (0-8) to make your move:")
    print("""
     0 | 1 | 2 
    ---+---+---
     3 | 4 | 5 
    ---+---+---
     6 | 7 | 8 
    """)
    while True:
        menace.play_game()
        play_again = input("Play again? (y/n): ").strip().lower()
        if play_again != "y":
            print("Thanks for playing!")
            break


In [ ]:
import random
import pickle
from collections import defaultdict

class MENACE:
    def __init__(self, model_file="menace_model.pkl"):
        # Dictionary to store state-action weights (equivalent to matchboxes and beads)
        self.states = defaultdict(lambda: defaultdict(int))  
        self.epsilon = 0.1  # Exploration probability
        self.model_file = model_file  # File to store the model

        # Load the previously saved model if exists
        self.load_model()

    def choose_action(self, state):
        # MENACE selects an action based on epsilon-greedy policy
        actions = self.states[state]
        valid_actions = self.get_valid_actions(state)
        if random.random() < self.epsilon or not actions:
            return random.choice(valid_actions)  # Exploration
        return max(valid_actions, key=lambda action: actions[action])  # Exploitation

    def update_weights(self, history, result):
        # Reinforce the moves based on the game result
        for state, action in history:
            if result == "win":
                self.states[state][action] += 1  # Reward for winning moves
            elif result == "loss":
                self.states[state][action] -= 1  # Penalize losing moves

    @staticmethod
    def get_valid_actions(state):
        # Return a list of empty cells (valid moves)
        return [i for i, cell in enumerate(state) if cell == 0]

    @staticmethod
    def check_winner(state):
        # Check if there's a winner
        winning_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
            [0, 4, 8], [2, 4, 6],  # Diagonals
        ]
        for combo in winning_combinations:
            line = [state[i] for i in combo]
            if all(x == 1 for x in line):  # MENACE wins
                return "MENACE"
            if all(x == -1 for x in line):  # User wins
                return "User"
        return None

    @staticmethod
    def print_board(state):
        # Print the Tic-Tac-Toe board
        symbols = [" " if x == 0 else "X" if x == 1 else "O" for x in state]
        board = f"""
         {symbols[0]} | {symbols[1]} | {symbols[2]} 
        ---+---+---
         {symbols[3]} | {symbols[4]} | {symbols[5]} 
        ---+---+---
         {symbols[6]} | {symbols[7]} | {symbols[8]} 
        """
        print(board)

    def play_game(self):
        # Initialize the game state
        state = [0] * 9  # Empty board
        history = []  # Store MENACE's moves for reinforcement
        while True:
            # MENACE's turn
            action = self.choose_action(tuple(state))
            history.append((tuple(state), action))
            state[action] = 1  # MENACE plays as "X"
            self.print_board(state)

            # Check if MENACE wins or if the board is full
            winner = self.check_winner(state)
            if winner:
                print(f"{winner} wins!")
                self.update_weights(history, "win" if winner == "MENACE" else "loss")
                self.save_model()  # Save model after each game
                break
            if not self.get_valid_actions(state):
                print("It's a draw!")
                self.save_model()  # Save model after each game
                break

            # User's turn
            while True:
                try:
                    user_action = int(input("Enter your move (0-8): "))
                    if user_action in self.get_valid_actions(state):
                        state[user_action] = -1  # User plays as "O"
                        break
                    else:
                        print("Invalid move. Try again.")
                except ValueError:
                    print("Please enter a number between 0 and 8.")

            # Check if the user wins
            winner = self.check_winner(state)
            if winner:
                self.print_board(state)
                print(f"{winner} wins!")
                self.update_weights(history, "win" if winner == "User" else "loss")
                self.save_model()  # Save model after each game
                break
            if not self.get_valid_actions(state):
                print("It's a draw!")
                self.save_model()  # Save model after each game
                break

    def save_model(self):
        # Save the MENACE model (state-action weights) to a file
        with open(self.model_file, 'wb') as f:
            pickle.dump(self.states, f)
        print("Model saved.")

    def load_model(self):
        # Load the MENACE model from a file if it exists
        try:
            with open(self.model_file, 'rb') as f:
                self.states = pickle.load(f)
            print("Model loaded.")
        except FileNotFoundError:
            print("No previous model found. Starting fresh.")

# Main loop
if __name__ == "__main__":
    menace = MENACE()
    print("Welcome to MENACE Tic-Tac-Toe!")
    print("You are 'O'. MENACE is 'X'. Enter a number (0-8) to make your move:")
    print("""
     0 | 1 | 2 
    ---+---+---
     3 | 4 | 5 
    ---+---+---
     6 | 7 | 8 
    """)
    while True:
        menace.play_game()
        play_again = input("Play again? (y/n): ").strip().lower()
        if play_again != "y":
            print("Thanks for playing!")
            break
